In [ ]:
!git clone https://github.com/SamiaKabir/ChatGPT-Answers-to-SO-questions.git

In [ ]:
!pip install python-docx -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 2.9 MB/s eta 0:00:00


In [ ]:
from docx import Document
doc = Document('/content/ChatGPT-Answers-to-SO-questions/ChatGPT answers to SO questions/Labeler1/Annotations_new_1500_1.docx')


In [ ]:
def get_title_link(cell):
  try:
    title, url = cell.text.split('http')
    title = title.replace('\n', '')
    url = ('http' + url).replace('\n', '')
  except:
    title, url = cell.text, None
  return title, url

def get_annotations(cell):
  annotation = {}

  highlight = ""
  for paragraph in cell.paragraphs:
      for run in paragraph.runs:
          if run.font.highlight_color:
              highlight += run.text

          if highlight and run.font.color.rgb.__str__() != '000000':
            annotation[highlight] = run.text[1:-1].split(',')
            highlight = ""

  return annotation

def get_overall_labels(cell):
  scores = []
  useful = 0
  for paragraphs in cell.paragraphs:
    text = paragraphs.text
    if len(text) != 0:
      if 'Usefulness' in text:
        useful = int(text[text.find('(')+1:text.find(')')])
      elif text[0] != '-':
        scores.append(text)

  return useful, scores

In [ ]:
import os
import pandas as pd
path = '/content/ChatGPT-Answers-to-SO-questions/ChatGPT answers to SO questions/Labeler2'
files = os.listdir(path)

In [ ]:
for f in files:
  doc = Document(os.path.join(path, f))

  table = doc.tables[0]

  data = []

  keys = None
  for i, row in enumerate(table.rows):
      text = (cell.text for cell in row.cells)

      if i == 0:
          keys = tuple(text)
          continue

      row_data = dict(zip(keys, text))
      row_data['ID'] = row_data['ID'].replace('\n', '')

      title, url = get_title_link(row.cells[1])
      row_data['title'], row_data['url'] = title, url

      row_data['annotations'] = get_annotations(row.cells[2])

      useful, scores = get_overall_labels(row.cells[3])
      row_data['usefulness'] = useful
      row_data['labels'] = scores

      data.append(row_data)


  df = pd.DataFrame(data)
  df.to_csv('/content/drive/MyDrive/Sem3/ANLP/Project_Data/Labeler2/' + f[:-4] + 'csv')
  # break
    # break

Labels for Conciseness

In [ ]:
import os
os.listdir('/content/drive/MyDrive/Sem3/ANLP/Project_Data/Labeler1')

['Annotations_new_1500_2.csv',
 'Annotations_old_5000_2.csv',
 'Annotations_old_5000_3.csv',
 'Annotations_old_1500_3.csv',
 'Annotations_old_1500_1.csv',
 'Annotations_new_61_2.csv',
 'Annotations_old_61_1.csv',
 'Annotations_new_5000_1.csv',
 'Annotations_new_1500_1.csv',
 'Annotations_new_61_3.csv',
 'Annotations_new_1500_2.gsheet',
 'Copy of Annotations_new_1500_2.gsheet',
 'Annotations_new_61_2.gsheet']

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Sem3/ANLP/Project_Data/Labeler1/Annotations_new_1500_2.csv')
df.head()

In [ ]:
from ast import literal_eval
map_labels = {}

def get_labels(t):

  s = literal_eval(t)
  if s != ['Skip: no credible accepted answer']:
    for x in s:
      if x in map_labels:
        map_labels[x] += 1
      else:
        map_labels[x] = 1

  return t

for files in os.listdir('/content/drive/MyDrive/Sem3/ANLP/Project_Data/Labeler1'):
  try:
    df = pd.read_csv(os.path.join('/content/drive/MyDrive/Sem3/ANLP/Project_Data/Labeler1', files))
    df['labels'] = df['labels'].map(get_labels)
  except Exception as e:
    print(e)
    continue

[Errno 95] Operation not supported: '/content/drive/MyDrive/Sem3/ANLP/Project_Data/Labeler1/Annotations_new_1500_2.gsheet'
[Errno 95] Operation not supported: '/content/drive/MyDrive/Sem3/ANLP/Project_Data/Labeler1/Copy of Annotations_new_1500_2.gsheet'
[Errno 95] Operation not supported: '/content/drive/MyDrive/Sem3/ANLP/Project_Data/Labeler1/Annotations_new_61_2.gsheet'


In [ ]:
map_labels = {}


In [ ]:
def get_binary_labels(t):
  s = literal_eval(t)
  for x in s:
    if 'Concise' in x:
      return 1 if x == 'Concise' else 0

  return -1

concise_mapping = {'Not Concise': 1, 'Partially Concise': 2, 'Mostly Concise': 3, 'Concise': 4}

def get_four_score_labels(t):
  s = literal_eval(t)
  for x in s:
    if 'Concise' in x:
      try:
        return concise_mapping[x]
      except:
        continue

  return -1

for files in os.listdir('/content/drive/MyDrive/Sem3/ANLP/Project_Data/Labeler2'):
  try:
    df = pd.read_csv(os.path.join('/content/drive/MyDrive/Sem3/ANLP/Project_Data/Labeler2', files))
    df['labels'] = df['labels'].map(get_labels)
    df['binary_concise'] = df['labels'].map(get_binary_labels)
    df['four_score_concise'] = df['labels'].map(get_four_score_labels)
    df.to_csv(os.path.join('/content/drive/MyDrive/Sem3/ANLP/Project_Data/Labeler2', files))
  except Exception as e:
    print(e)
    continue

# df = pd.read_csv('/content/drive/MyDrive/Sem3/ANLP/Project_Data/Labeler1/Annotations_new_1500_2.csv')
# df['binary_concise'] = df['labels'].map(get_binary_labels)
# df['four_score_concise'] = df['labels'].map(get_four_score_labels)

In [ ]:
df.head()

,Unnamed: 0,ID,Title and post link,ChatGPT Answer,Overall Labels,title,url,annotations,usefulness,labels,binary_concise,four_score_concise
0,0,75251315,Dependency issue installing google-chrome-stab...,The issue you're encountering is due to a miss...,Partially Comprehensive\n\nUsefulness (4)\n---...,Dependency issue installing google-chrome-stab...,https://stackoverflow.com/questions/75251315/d...,{'The issue you\'re encountering is due to a m...,4,"['Partially Comprehensive', 'Correct', 'Incons...",0,1
1,1,75254685,GPG error: https://apt.releases.hashicorp.com ...,This error typically occurs when the apt packa...,Fully Comprehensive\n\nUsefulness (4)\n-------...,GPG error: https://apt.releases.hashicorp.com ...,NaN,{'1. Identify the missing public key: Take not...,4,"['Fully Comprehensive', 'Correct', 'Inconsiste...",1,4
2,2,75263047,Duplicate class in Kotlin Android\n\nhttps://s...,The error you're facing is caused by conflicti...,Partially Comprehensive\n\nUsefulness (4)\n---...,Duplicate class in Kotlin Android,https://stackoverflow.com/questions/75263047/d...,"{""The error you're facing is caused by conflic...",4,"['Partially Comprehensive', 'Correct', 'Incons...",0,1
3,3,75272737,"Error ""Invalid version: '0.23ubuntu1' (package...",It seems that you're encountering multiple iss...,Fully Comprehensive\n\nUsefulness (2)\n-------...,"Error ""Invalid version: '0.23ubuntu1' (package...",https://stackoverflow.com/questions/75272737/e...,{'1. Error building `pycairo` wheel:Since you ...,2,"['Fully Comprehensive', 'Partially Correct', '...",0,1
4,4,75275563,AttributeError: module 'sqlalchemy' has no att...,"The initial error you encountered, `AttributeE...",Fully Comprehensive\n\nUsefulness (5)\n-------...,AttributeError: module 'sqlalchemy' has no att...,https://stackoverflow.com/questions/75275563/a...,{'5. Verify GitHub Actions environment: Double...,5,"['Fully Comprehensive', 'Correct', 'Consistent...",0,1


In [ ]:
df.iloc[3,-1]

4

In [ ]:
df.iloc[3,-3]

"['Fully Comprehensive', 'Partially Correct', 'Partially Consistent', 'Concise']"

In [ ]:
df['four_score_concise'].value_counts()

 1    17
 4    13
-1     4
Name: four_score_concise, dtype: int64

In [ ]:
df['binary_concise'].value_counts()

 0    17
 1    13
-1     4
Name: binary_concise, dtype: int64

In [ ]:
map_labels = {}
df['labels'] = df['labels'].map(get_labels)
map_labels

{'Fully Comprehensive': 17,
 'Partially Correct': 4,
 'Partially Consistent': 4,
 'Concise': 2,
 'Incorrect': 8,
 'Inconsistent': 9,
 'Correct': 8,
 'Partially Inconsistent': 3,
 'Mostly Concise': 6,
 'Partially Comprehensive': 4,
 'Not Concise': 8,
 'Partially Concise': 7,
 'Mostly Inconsistent': 4,
 'Consistent': 2,
 'Not Comprehensive': 1,
 'Mostly Incorrect': 3,
 'Mostly Consistent': 1,
 'Partially\nComprehensive': 1}